In [1]:
import pandas as pd
import numpy as np
from datetime import datetime 
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import metrics
from sklearn. preprocessing import PolynomialFeatures
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import gc
import xgboost

C:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
lines = ['1', '13', '37', '104', '11', '102', '151', '41', '65', '7A', '140', '84', '79A', '15A', '17', '44', '40B', '41C', '83A', '238', 
         '25', '38A', '63', '4', '66A', '31', '150', '17A', '25B', '31A', '33', '27B', '32', '14', '67', '9', '61', '43', '130', '68', 
         '15', '123', '122', '120', '47', '69', '39A', '76', '40', '70', '29A', '25A', '38', '49', '65B', '39', '45A', '42', '77A', 
         '27', '54A', '46A', '27A', '40D', '7', '53', '33B', '75', '270', '18', '16', '41B', '145', '83', '79', '16C', '66', '15B', 
         '185', '26', '56A', '33A', '14C', '59', '220', '184', '142', '114', '76A', '31B', '68A', '66B', '111', '236', '84A', '67X', 
         '84X', '239', '42D', '66X', '116', '41X', '46E', '38B', '32X', '161', '7B', '118', '25X', '38D', '7D', '44B', '33X', '25D', 
         '51D', '15D', '31D', '39X', '27X', '51X', '77X', '69X', '41A', '70D', '68X', '33D', '40E', '16D', '33E', '41D']

In [3]:
line = '41D'
# line = '47'
Line_47 = pd.read_csv('Step_1/'+line+'.csv',sep=',')
Line_47

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION
0,11-DEC-18 00:00:00,8173538,13,3699,33421,33421,1932358,41D,41D_27,2
1,11-DEC-18 00:00:00,8173538,12,3698,33407,33407,1932358,41D,41D_27,2
2,11-DEC-18 00:00:00,8173538,11,3697,33369,33369,1932358,41D,41D_27,2
3,11-DEC-18 00:00:00,8173538,10,3696,33346,33346,1932358,41D,41D_27,2
4,11-DEC-18 00:00:00,8173538,9,3695,33301,33301,1932358,41D,41D_27,2
...,...,...,...,...,...,...,...,...,...,...
1112,17-DEC-18 00:00:00,8463181,22,1633,34023,34023,1932361,41D,41D_27,2
1113,17-DEC-18 00:00:00,8463181,39,45,35144,35173,1932361,41D,41D_27,2
1114,18-DEC-18 00:00:00,8463182,1,5044,62405,62405,1001166,41D,41D_27,2
1115,18-DEC-18 00:00:00,8463182,10,3696,62687,62709,1001166,41D,41D_27,2


In [4]:
Line_47.corr()

,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,DIRECTION
TRIPID,1.000000,0.076736,-0.025363,0.000802,0.000789,-0.010725,0.093079
PROGRNUMBER,0.076736,1.000000,0.030316,0.336152,0.336013,-0.233291,0.255315
STOPPOINTID,-0.025363,0.030316,1.000000,-0.013963,-0.014162,0.043796,0.045709
ACTUALTIME_ARR,0.000802,0.336152,-0.013963,1.000000,0.999999,-0.777624,0.793786
ACTUALTIME_DEP,0.000789,0.336013,-0.014162,0.999999,1.000000,-0.777651,0.793683
VEHICLEID,-0.010725,-0.233291,0.043796,-0.777624,-0.777651,1.000000,-0.561511
DIRECTION,0.093079,0.255315,0.045709,0.793786,0.793683,-0.561511,1.000000


In [5]:
new = Line_47.DIRECTION.rename('time')
daytime = Line_47.DIRECTION.rename('DAYTIME')
date = Line_47.DIRECTION.rename('DATE')

In [6]:
for i in range(Line_47.shape[0]):
    d = datetime.strptime(Line_47.DAYOFSERVICE[i],'%d-%b-%y %H:%M:%S')
    hour = Line_47.ACTUALTIME_ARR[i]//3600
    minute = (Line_47.ACTUALTIME_ARR[i]-hour*3600)//60
    new[i] = hour*3600+(minute//10)*10*60
    daytime[i] = d + timedelta(hours=int(hour))
    date[i] = d
#     print(daytime[i],new[i])
    print(i,end=' ')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [7]:
Line_47_new = pd.concat([Line_47,daytime.rename('DAYTIME'),new.rename("TIME"),date.rename('DATE')],axis = 1)
Line_47_new = Line_47_new.drop(['ROUTEID'],axis=1)
Line_47_new.DAYTIME = pd.to_datetime(daytime)
Line_47_new

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,DIRECTION,DAYTIME,TIME,DATE
0,11-DEC-18 00:00:00,8173538,13,3699,33421,33421,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00
1,11-DEC-18 00:00:00,8173538,12,3698,33407,33407,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00
2,11-DEC-18 00:00:00,8173538,11,3697,33369,33369,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00
3,11-DEC-18 00:00:00,8173538,10,3696,33346,33346,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00
4,11-DEC-18 00:00:00,8173538,9,3695,33301,33301,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1112,17-DEC-18 00:00:00,8463181,22,1633,34023,34023,1932361,41D,2,2018-12-17 09:00:00,33600,2018-12-17 00:00:00
1113,17-DEC-18 00:00:00,8463181,39,45,35144,35173,1932361,41D,2,2018-12-17 09:00:00,34800,2018-12-17 00:00:00
1114,18-DEC-18 00:00:00,8463182,1,5044,62405,62405,1001166,41D,2,2018-12-18 17:00:00,62400,2018-12-18 00:00:00
1115,18-DEC-18 00:00:00,8463182,10,3696,62687,62709,1001166,41D,2,2018-12-18 17:00:00,62400,2018-12-18 00:00:00


In [8]:
# low = '2018-01-01 08:00:00'
# high = '2018-02-01 08:00:00'
# Line_47_subset = Line_47_new[(Line_47_new.DAYTIME>=low) & (Line_47_new.DAYTIME<=high)]

In [9]:
# low = '2018-04-01 08:00:00'
# high = '2018-05-01 08:00:00'
# Line_47_subset = pd.concat([Line_47_subset, Line_47_new[(Line_47_new.DAYTIME>=low) & (Line_47_new.DAYTIME<=high)]])

In [10]:
# low = '2018-07-01 08:00:00'
# high = '2018-08-01 08:00:00'
# Line_47_subset = pd.concat([Line_47_subset, Line_47_new[(Line_47_new.DAYTIME>=low) & (Line_47_new.DAYTIME<=high)]])

In [11]:
# low = '2018-10-01 08:00:00'
# high = '2018-11-01 08:00:00'
# Line_47_subset = pd.concat([Line_47_subset, Line_47_new[(Line_47_new.DAYTIME>=low) & (Line_47_new.DAYTIME<=high)]])

In [12]:
# Line_47_new = Line_47_subset
# Line_47_new

In [13]:
Line_47_new["ORDER"] = Line_47_new.TIME

In [14]:
orders = Line_47_new.PROGRNUMBER.unique()
orders

array([13, 12, 11, 10,  9,  8,  1, 46, 45, 43, 42, 41, 40, 39, 38, 37, 36,
       35, 34, 32, 31, 30, 29, 28, 27, 26, 25, 24, 22, 14,  7, 33, 21, 20,
       19, 17, 15,  6,  4,  3,  2, 23, 18, 16,  5, 44, 47], dtype=int64)

In [15]:
Line_47_new = Line_47_new.reset_index(drop=True)
Line_47_new

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,DIRECTION,DAYTIME,TIME,DATE,ORDER
0,11-DEC-18 00:00:00,8173538,13,3699,33421,33421,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00,33000
1,11-DEC-18 00:00:00,8173538,12,3698,33407,33407,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00,33000
2,11-DEC-18 00:00:00,8173538,11,3697,33369,33369,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00,33000
3,11-DEC-18 00:00:00,8173538,10,3696,33346,33346,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00,33000
4,11-DEC-18 00:00:00,8173538,9,3695,33301,33301,1932358,41D,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00,33000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,17-DEC-18 00:00:00,8463181,22,1633,34023,34023,1932361,41D,2,2018-12-17 09:00:00,33600,2018-12-17 00:00:00,33600
1113,17-DEC-18 00:00:00,8463181,39,45,35144,35173,1932361,41D,2,2018-12-17 09:00:00,34800,2018-12-17 00:00:00,34800
1114,18-DEC-18 00:00:00,8463182,1,5044,62405,62405,1001166,41D,2,2018-12-18 17:00:00,62400,2018-12-18 00:00:00,62400
1115,18-DEC-18 00:00:00,8463182,10,3696,62687,62709,1001166,41D,2,2018-12-18 17:00:00,62400,2018-12-18 00:00:00,62400


In [16]:
Line_47_withOrder = pd.DataFrame(columns=['DAYOFSERVICE', 'TRIPID', 'PROGRNUMBER', 'STOPPOINTID',
       'ACTUALTIME_ARR', 'ACTUALTIME_DEP', 'VEHICLEID', 'DIRECTION', 'DAYTIME',
       'TIME', 'DATE', 'ORDER'])
for i in range(100):
    for j in orders:
        df = Line_47_new[(Line_47_new.DATE == Line_47_new.DATE[0]+timedelta(days=i))&(Line_47_new.DIRECTION==1)&(Line_47_new.PROGRNUMBER==int(j))].sort_values('ACTUALTIME_ARR')
        df_2 = Line_47_new[(Line_47_new.DATE == Line_47_new.DATE[0]+timedelta(days=i))&(Line_47_new.DIRECTION==2)&(Line_47_new.PROGRNUMBER==int(j))].sort_values('ACTUALTIME_ARR')
        df = df.reset_index(drop=True)
        df_2 = df_2.reset_index(drop=True)
        for k in range(df.shape[0]):
            df.ORDER[k] = k+1
#             print(df.ORDER[k],end=' ')
        Line_47_withOrder = pd.concat([Line_47_withOrder,df],axis=0)
        for k in range(df_2.shape[0]):
            df_2.ORDER[k] = k+1
#             print(df.ORDER[k],end=' ')
        Line_47_withOrder = pd.concat([Line_47_withOrder,df_2],axis=0)
    print(i,end=' ')

# df = Line_47_new[(Line_47_new.DATE == Line_47_new.DATE[0]+timedelta(days=0))&(Line_47_new.DIRECTION==1)&(Line_47_new.PROGRNUMBER==1)].sort_values('ACTUALTIME_ARR')
# df = df.reset_index(drop=True)
# df


C:\anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [17]:
# for i in range(len(Line_47_new.shape[0])):
#     df = Line_47_new[(Line_47_new.TRIPID ==Line_47_new.TRIPID[i]) & (Line_47_new.DAYOFSERVICE == Line_47_new.DAYOFSERVICE[i])].sort_values(by='PROGRNUMBER')
#     for j in df.shape[0]:
#         if df.PROGRNUMBER > Line_47_new.PROGRNUMBER[i]:
            

In [18]:
# Line_47_withOrder.to_csv('Line_47_withOrder.csv',header=True,index=False)
# # Line_47_withOrder = pd.read_csv('Line_47_withOrder.csv',sep=',')
# Line_47_withOrder

In [19]:
Line_47_withOrder

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,DIRECTION,DAYTIME,TIME,DATE,ORDER,LINEID
0,11-DEC-18 00:00:00,8168184,13,85,27968,27979,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D
1,11-DEC-18 00:00:00,8173537,13,85,29528,29542,1932358,1,2018-12-11 08:00:00,29400,2018-12-11 00:00:00,2,41D
0,11-DEC-18 00:00:00,8173538,13,3699,33421,33421,1932358,2,2018-12-11 09:00:00,33000,2018-12-11 00:00:00,1,41D
1,11-DEC-18 00:00:00,8172053,13,3699,62909,62909,1001153,2,2018-12-11 17:00:00,62400,2018-12-11 00:00:00,2,41D
0,11-DEC-18 00:00:00,8168184,12,7602,27909,27909,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,21-DEC-18 00:00:00,8463181,5,3690,33120,33120,2693256,2,2018-12-21 09:00:00,33000,2018-12-21 00:00:00,1,41D
0,21-DEC-18 00:00:00,8463181,44,51,35589,35596,2693256,2,2018-12-21 09:00:00,35400,2018-12-21 00:00:00,1,41D
1,21-DEC-18 00:00:00,8463182,44,51,64475,64475,1000196,2,2018-12-21 17:00:00,64200,2018-12-21 00:00:00,2,41D
0,21-DEC-18 00:00:00,8463181,47,288,36063,36063,2693256,2,2018-12-21 10:00:00,36000,2018-12-21 00:00:00,1,41D


In [20]:
Line_47_withOrder[Line_47_withOrder.STOPPOINTID==1512]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,DIRECTION,DAYTIME,TIME,DATE,ORDER,LINEID


In [21]:
Line_47_withOrder[Line_47_withOrder.STOPPOINTID==1538]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,DIRECTION,DAYTIME,TIME,DATE,ORDER,LINEID


In [22]:
# a = list(Line_47_new.VEHICLEID.unique())
# a.sort()
# print(a)
Line_47_withOrder = Line_47_withOrder.drop(['DAYOFSERVICE','TRIPID','ACTUALTIME_DEP'],axis=1)

In [23]:
weather = pd.read_csv('Weather_ReadytoMerge.csv',sep=',')
weather.DAYTIME = pd.to_datetime(weather.DAYTIME)
weather

,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,DAYTIME,DAYOFWEEK
0,0.0,-6.49,3.84,5.79,990,87,12.86,240,40,520,7,2018-01-01 00:00:00,0.0
1,0.0,-5.79,3.65,5.86,991,87,11.83,240,75,520,7,2018-01-01 01:00:00,0.0
2,0.0,-5.77,3.85,5.99,990,81,12.35,240,40,802,5,2018-01-01 02:00:00,0.0
3,0.0,-5.73,4.00,6.14,990,81,12.35,240,40,802,5,2018-01-01 03:00:00,0.0
4,0.0,-4.91,4.11,6.22,990,81,11.83,240,40,802,5,2018-01-01 04:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,0.0,5.78,8.03,10.00,1034,76,4.10,260,75,803,5,2018-12-31 19:00:00,0.0
8796,0.0,5.52,8.22,10.00,1034,81,4.10,250,75,803,5,2018-12-31 20:00:00,0.0
8797,0.0,4.87,8.33,10.00,1034,81,5.10,260,75,803,5,2018-12-31 21:00:00,0.0
8798,0.0,4.30,8.33,10.00,1035,81,5.70,260,75,803,5,2018-12-31 22:00:00,0.0


In [24]:
weather.DAYTIME = pd.to_datetime(weather.DAYTIME)
Line_47_withOrder.DAYTIME = pd.to_datetime(Line_47_withOrder.DAYTIME)

In [25]:
dataset = pd.merge(Line_47_withOrder,weather,on='DAYTIME')
dataset.to_csv('Step_2/dataset_'+line+'.csv',header=True,index=False)
dataset

,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,VEHICLEID,DIRECTION,DAYTIME,TIME,DATE,ORDER,LINEID,...,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,DAYOFWEEK
0,13,85,27968,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
1,12,7602,27909,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
2,11,21,27854,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
3,10,19,27784,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
4,9,18,27770,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,23,1634,63182,1000196,2,2018-12-21 17:00:00,63000,2018-12-21 00:00:00,2,41D,...,7.11,9.00,999,81,5.1,230,75,803,5,4.0
1149,44,51,64475,1000196,2,2018-12-21 17:00:00,64200,2018-12-21 00:00:00,2,41D,...,7.11,9.00,999,81,5.1,230,75,803,5,4.0
1150,46,277,64851,1000196,2,2018-12-21 18:00:00,64800,2018-12-21 00:00:00,2,41D,...,7.13,9.00,999,75,5.1,240,75,803,5,4.0
1151,47,288,64999,1000196,2,2018-12-21 18:00:00,64800,2018-12-21 00:00:00,2,41D,...,7.13,9.00,999,75,5.1,240,75,803,5,4.0


# ---------------------------------------------------------------------------------------------------------------

In [26]:
dataset = pd.read_csv('Step_2/dataset_'+line+'.csv',sep=',')
dataset

,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,VEHICLEID,DIRECTION,DAYTIME,TIME,DATE,ORDER,LINEID,...,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,DAYOFWEEK
0,13,85,27968,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
1,12,7602,27909,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
2,11,21,27854,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
3,10,19,27784,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
4,9,18,27770,2534849,1,2018-12-11 07:00:00,27600,2018-12-11 00:00:00,1,41D,...,5.00,9.62,1019,93,1.5,0,75,701,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,23,1634,63182,1000196,2,2018-12-21 17:00:00,63000,2018-12-21 00:00:00,2,41D,...,7.11,9.00,999,81,5.1,230,75,803,5,4.0
1149,44,51,64475,1000196,2,2018-12-21 17:00:00,64200,2018-12-21 00:00:00,2,41D,...,7.11,9.00,999,81,5.1,230,75,803,5,4.0
1150,46,277,64851,1000196,2,2018-12-21 18:00:00,64800,2018-12-21 00:00:00,2,41D,...,7.13,9.00,999,75,5.1,240,75,803,5,4.0
1151,47,288,64999,1000196,2,2018-12-21 18:00:00,64800,2018-12-21 00:00:00,2,41D,...,7.13,9.00,999,75,5.1,240,75,803,5,4.0


In [27]:
dataset.corr()

,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,VEHICLEID,DIRECTION,TIME,ORDER,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,DAYOFWEEK
PROGRNUMBER,1.000000,0.004342,0.327868,-0.219318,0.258082,0.327758,-0.103000,-0.014871,0.079606,0.144015,0.104565,-0.082841,0.031916,0.026593,0.058255,0.056227,-0.074415,0.085377,-0.014871
STOPPOINTID,0.004342,1.000000,-0.015197,0.043435,0.045181,-0.015330,-0.000405,0.022013,-0.027823,-0.012939,-0.029661,0.013378,0.002300,-0.003816,-0.006682,-0.018701,-0.033329,0.026352,0.022013
ACTUALTIME_ARR,0.327868,-0.015197,1.000000,-0.776756,0.764301,0.999931,0.043294,-0.007499,0.024944,0.378665,0.261940,-0.035719,-0.085001,0.362411,0.068770,0.089916,-0.082587,0.151815,-0.007499
VEHICLEID,-0.219318,0.043435,-0.776756,1.000000,-0.518247,-0.776853,-0.000899,0.314179,-0.280543,-0.252520,-0.324002,-0.153809,-0.103290,-0.011713,0.119706,-0.201273,-0.092920,0.054830,0.314179
DIRECTION,0.258082,0.045181,0.764301,-0.518247,1.000000,0.764412,-0.225214,0.017796,0.035997,0.289762,0.202034,-0.119012,-0.063676,0.258398,0.158567,0.148848,-0.142610,0.177183,0.017796
TIME,0.327758,-0.015330,0.999931,-0.776853,0.764412,1.000000,0.042632,-0.007030,0.025342,0.378635,0.262018,-0.035956,-0.084190,0.361893,0.069213,0.090126,-0.083023,0.152003,-0.007030
ORDER,-0.103000,-0.000405,0.043294,-0.000899,-0.225214,0.042632,1.000000,-0.058868,0.088682,0.076353,0.090567,0.003270,-0.047011,-0.039833,-0.042860,-0.116725,0.094383,-0.051250,-0.058868
temp,-0.014871,0.022013,-0.007499,0.314179,0.017796,-0.007030,-0.058868,1.000000,-0.524054,-0.333680,-0.585689,-0.152143,0.138907,0.175013,0.473973,-0.005771,-0.442174,0.353111,1.000000
feels_like,0.079606,-0.027823,0.024944,-0.280543,0.035997,0.025342,0.088682,-0.524054,1.000000,0.510099,0.774783,-0.143554,0.505506,-0.622264,-0.054837,0.075642,0.173208,-0.288899,-0.524054
temp_min,0.144015,-0.012939,0.378665,-0.252520,0.289762,0.378635,0.076353,-0.333680,0.510099,1.000000,0.862528,-0.122208,-0.064107,0.317741,-0.046123,0.231073,-0.036433,0.189591,-0.333680


In [28]:
dataset.DIRECTION.unique()

array([1, 2], dtype=int64)

In [29]:
dataset.DAYTIME = pd.to_datetime(dataset.DAYTIME)

In [30]:
train_features = ['STOPPOINTID','DAYOFWEEK','PROGRNUMBER','ORDER','temp','feels_like',
        'pressure', 'humidity', 'wind_speed','wind_deg', 'clouds_all', 'weather_id', 'weather_main']
target_features = ['ACTUALTIME_ARR']

In [31]:
# Xtrain,Xtest,Ytrain,Ytest = train_test_split(dataset[train_features],dataset[target_features],test_size = 0.3)
Xtrain,Xtest,Ytrain,Ytest = train_test_split(dataset[train_features],dataset[target_features],test_size = 0.3)

In [32]:
reg = xgboost.XGBRegressor(
                            booster='gbtree'
                            ,max_depth=19
                            )
reg = reg.fit(Xtrain,Ytrain)
score = reg.score(Xtest,Ytest)
score

[23:01:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9313815788750234

In [33]:
reg.feature_importances_

array([1.0458082e-03, 2.2585023e-02, 8.2307924e-03, 7.6880213e-04,
       0.0000000e+00, 5.4277390e-02, 2.5194058e-02, 1.5536478e-01,
       2.1063554e-01, 1.6205718e-03, 1.7268655e-01, 3.4758428e-01,
       6.4245892e-06], dtype=float32)

In [34]:
# rfc = RandomForestRegressor(
#                             random_state=0
#                             ,n_estimators=20
#                             ,max_depth=25
#                            )
# rfc = rfc.fit(Xtrain,Ytrain)
# score =rfc.score(Xtest,Ytest)
# score

In [35]:
# rfc.feature_importances_

In [36]:
def printMetrics(testActualVal, predictions): 
    #classification evaluation measures
    print('\n==============================================================================') 
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions)) 
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions)) 
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5) 
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [37]:
test_prediction = reg.predict(Xtest)
printMetrics(Ytest,test_prediction)


MAE:  727.5235447525289
RMSE:  3857.058507554308
R2:  0.9313815788750235


In [38]:
joblib.dump(reg,'Models/Model_'+line+'.joblib')

['Models/Model_41D.joblib']

In [39]:
# d = pd.DataFrame([[374,0,15,1,5.47,5.47,992, 87, 7.72,240, 40, 802, 5]])
# d.columns =  ['STOPPOINTID','DAYOFWEEK','PROGRNUMBER','ORDER','temp','feels_like',
#         'pressure', 'humidity', 'wind_speed','wind_deg', 'clouds_all', 'weather_id', 'weather_main']
# d

In [40]:
# reg.predict(d)

In [41]:
# df = pd.read_table('rt_trips_DB_2018.txt',sep=';')

In [42]:
# df.LINEID.unique()

In [43]:
# train = ['DAYOFWEEK','TIME','STOPPOINTID']
# target = ['ORDER']

In [44]:
# xtrain,xtest,ytrain,ytest = train_test_split(dataset[train],dataset[target],test_size = 0.3)

In [45]:
# del xtrain,xtest,ytrain,ytest

In [46]:
# gc.collect()

In [47]:
# rfc_1 = RandomForestRegressor(
#                             random_state=0
#                             ,n_estimators=20
#                             )
# rfc_1 = rfc.fit(xtrain,ytrain)
# score = rfc.score(xtest,ytest)
# score

In [48]:
# test_prediction = rfc_1.predict(xtest)
# printMetrics(ytest,test_prediction)

In [49]:
# joblib.dump(rfc_1,'prediceOrder.joblib')

In [50]:
# df = pd.read_table('rt_vehicles_DB_2018.txt',sep=';')
# df

In [51]:
# t = ['MINUTES','DAYOFWEEK']
# xtrain,xtest,ytrain,ytest = train_test_split(df[t],df['VEHICLEID'],test_size = 0.3)

In [52]:
# new = df.DATASOURCE
# for i in range(df.shape[0]):
#     d = datetime.strptime(df.DAYOFSERVICE[i],'%d-%b-%y %H:%M:%S')
#     new[i] = d
#     print(i,end=' ')

In [53]:
# df = pd.concat([df,new.rename('DAYTIME')],axis=1)
# # df = df.drop(['DATASOURCE'],axis=1)
# df.DAYTIME = pd.to_datetime(df.DAYTIME)
# df = pd.merge(df,weather,on='DAYTIME')
# df


In [54]:
l = Line_47_new[Line_47_new.STOPPOINTID==374]

In [55]:
sum(l.ACTUALTIME_ARR)/l.shape[0]

ZeroDivisionError: division by zero

In [ ]:
l = l.reset_index()
l

In [ ]:
err = 0
for i in range(l.shape[0]):
    err = err + abs(int(l.ACTUALTIME_ARR[i]) - 57954.244352236055)
    print(l.ACTUALTIME_ARR[i],' ',abs(int(l.ACTUALTIME_ARR[i]) - 57954.244352236055))
print('MAE=',err/l.shape[0])